<a href="https://colab.research.google.com/github/srinithish/product-recognition/blob/master/Confidence_Loss_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np
import sklearn
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import random
import matplotlib.cm as cm
import math
import pickle

In [12]:

from google.colab import drive
drive.mount('/content/Drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/Drive


In [0]:
# https://medium.com/@jonathan_hui/real-time-object-detection-with-yolo-yolov2-28b1b93e2088
# https://github.com/thtrieu/darkflow/blob/master/darkflow/net/yolo/train.py
# https://hackernoon.com/understanding-yolo-f5a74bbc7967

def confidence_loss(y_true,y_pred):
    '''
    y_true.shape => (batch,grows,gcols,params)    \\ params = P(obj) + 4 b.b params + each class probabilities
    y_pred.shape => y_true.shape
        
    b.b params => (x,y,h,w)
    
    There are 3 types of losses:
    1. classification loss
    2. localization loss
    3. confidence loss
    
    The λ parameters that appear here and also in the first part are used to 
    differently weight parts of the loss functions. This is necessary to increase 
    model stability. The highest penalty is for coordinate predictions (λ coord = 5) 
    and the lowest for confidence predictions when no object is present (λ noobj = 0.5).
    '''
    
    ##### Confidence loss
    '''
    Most boxes do not contain any objects. This causes a class imbalance problem,
    i.e. we train the model to detect background more frequently than detecting 
    objects. To remedy this, we weight this loss down by a factor λnoobj (default: 0.5).
    box_conf_score = P(obj) * (IoU with g.t)
    '''
    lambda_noobj = 0.5
    object_mask = y_true[:,:,:,:1] # 1 where there is an object, 0 elsewhere
    no_object_mask = 1 - object_mask # Complement of object mask
    
    P_obj_pred = y_pred[:,:,:,:1]    # Slice probs
    P_obj_true = y_true[:,:,:,:1]
    
    # Calculate iou for all the predicted boxes
    
    # Pred area
    box_pred = y_pred[:,:,:,3:5]
    coords_pred = y_pred[:,:,:,1:3]
    
    denorm_coords_pred = Denormalize_coordinates(coords_pred) # Slice the x,y
    lefttop_pred,rightbot_pred = Corner_coords(denorm_coords_pred,box_pred)
    area_pred = Area(lefttop_pred,rightbot_pred)
    
    # True area
    box_true = y_true[:,:,:,3:5]
    coords_true = y_true[:,:,:,1:3]
    
    denorm_coords_true = Denormalize_coordinates(coords_true) # Slice the x,y
    lefttop_true,rightbot_true = Corner_coords(denorm_coords_true,box_true)
    area_true = Area(lefttop_true,rightbot_true)
    
    # Calculate intersection
    intersect = Intersection(lefttop_pred,rightbot_pred,lefttop_true,rightbot_true)
    
    # Calculate iou
    iou = tf.truediv(intersect, area_true + area_pred - intersect)  # intersection/union
    # iou.shape => (batch,grows,gcols,1)
    
    # Prediction confidences of the grid cell
    pred_confs = tf.math.multiply(P_obj_pred,iou) # confs = P*iou
    
    # G.T confidences
    true_confs = tf.math.multiply(P_obj_true,iou)
    
    # We want square of diff of True C - Pred C
    loss = tf.multiply(object_mask,tf.pow(true_confs - pred_confs, 2))
    total_loss = loss + (tf.multiply(no_object_mask,tf.pow(true_confs - pred_confs, 2)) * lambda_noobj)
    flat_loss = tf.layers.flatten(total_loss)
    batch_loss = tf.reduce_sum(flat_loss,axis=1)
    
    return tf.reduce_mean(batch_loss)

In [0]:
def Denormalize_coordinates(pred_coords):
    '''
    Inputs
    pred_coords.shape => (batch,grows,gcols,2) # normed x and y of the b.b
    
    Output:
    denormalized_coord.shape => (batch,grows,gcols,2) # denormed x and y of the b.b
    '''
    img_height = tf.constant(342)
    img_width = tf.constant(342)
    
    grows = tf.constant(19)
    gcols = tf.constant(19)
    
    cell_height = tf.truediv(img_height,grows)
    cell_width = tf.truediv(img_width,gcols)
    
    col_id = tf.to_float(tf.reshape(tf.tile(tf.range(grows), [gcols]), (1, grows, gcols, 1)))
    row_id = tf.to_float(tf.reshape(tf.tile(tf.range(grows), [gcols]), (1, grows, gcols, 1)))
    row_id = tf.transpose(row_id)
    coord_id = tf.concat([col_id,row_id],-1) # a 4D tensor that gives the coordinates for every box (col,row)

    coords_lefttop = tf.multiply(coord_id,[cell_width,cell_height]) # Gets the leftmost coordinate for every box
    cell_denormalized_coords = tf.multiply(pred_coords,[cell_width,cell_height])
    
    denormalized_coord = coords_lefttop + cell_denormalized_coords
    
    return denormalized_coord

In [0]:
def Corner_coords(denorm_coords,box_params):
    '''
    box_params are h and w
    We have [h,w] but we want [w,h]
    This is because the coords tensor has [x,y]
    
    box_params.shape => (batch,grows,gcols,2)
    
    Output:
    lefttop and rightbot points of this b.b
    '''
    img_height = tf.constant(342)
    img_width = tf.constant(342)
    
    grows = tf.constant(19)
    gcols = tf.constant(19)
    
    cell_height = tf.truediv(img_height,grows)
    cell_width = tf.truediv(img_width,gcols)
    
    box_params = tf.reverse(box_params,[-1]) # Now w,h
    box_params = tf.multiply(box_params,[img_width,img_height])
    
    halfs = box_params/2
    
    lefttop = denorm_coords - halfs
    rightbot = denorm_coords + halfs
    
    return lefttop,rightbot

In [0]:
def Area(lefttop,rightbot):
    diff = rightbot - lefttop
    
    area = tf.math.reduce_prod(diff,axis=-1,keepdims=True) # shape => (batch,grows,gcols,1)
    
    return area    

In [0]:
def Intersection(p1,p2,p3,p4):
    '''
    p1 is the lefttop point of the first b.b
    p2 is the rightbot point of the first b.b
    p3 is the lefttop point of the second b.b
    p4 is the rightbot point of the second b.b
    
    p5 => take the max of p1 and p3
    p6 => take the min of p2 and p4
    
    p5 and p6 are the intersection box's lefttop and righttop coords
    find area of this box and return it
    
    p1.shape => (batch,grwos,gcols,2)
    '''
    p5 = tf.math.maximum(p1,p3)
    p6 = tf.math.minimum(p2,p4)
    
    area = Area(p5,p6)
    return area
    
        

In [8]:
y_true = tf.ones([170,19,19,70],dtype=tf.dtypes.float32)
y_pred = tf.random.normal([170,19,19,70],dtype=tf.dtypes.float32)

tf.InteractiveSession()
y = y_pred.eval()
print(np.shape(y))
print(np.shape(y_pred[...,1].eval()))
loss = confidence_loss(y_true,tf.sigmoid(y_pred))

(170, 19, 19, 70)
(170, 19, 19)
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use keras.layers.flatten instead.


In [0]:
dirpathForTrainingFiles  = "/content/Drive/My Drive/Deeplearning Project/Reshaped/"



annotationsPath = dirpathForTrainingFiles + 'annotations/'
imagesPath = dirpathForTrainingFiles+ 'images/'

###for image as numpy array X
imagesAsArrayPklPath =  annotationsPath+'resizedAllImgArray.pkl'
###for true Y
targetVariablePklPath = annotationsPath+'resizedAllTargetArray.pkl'

### for visualising

imagefilePattern = imagesPath+'*'
ImgDictsPath_True_Path = annotationsPath+'resizedImageDictsAllFiles.pkl'
ObjLists_True_Path = annotationsPath+'resizedObjectListsAllFiles.pkl'
predictionArrayPath = annotationsPath+'PredictionArray.pkl'


TrainX = np.array(pickle.load(open(imagesAsArrayPklPath, 'rb')))

TrainY = np.array(pickle.load(open(targetVariablePklPath, 'rb')))
predictionsY = np.array(pickle.load(open(predictionArrayPath, 'rb')))
predictionsY = predictionsY.astype(np.float32)
TrainY = TrainY.astype(np.float32)

In [14]:

sess  = tf.InteractiveSession()
predictionsY = tf.convert_to_tensor(predictionsY)
TrainY = tf.convert_to_tensor(TrainY)

loss = confidence_loss(predictionsY,TrainY)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [15]:
loss.eval()

0.114670254